In [1]:
import utils
import config
from finetune import finetune_model
from transformers import BertForSequenceClassification, BertTokenizer

c:\Users\anast\AppData\Local\pypoetry\Cache\virtualenvs\tda-for-fairness-5qunVEZW-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PATH_TO_DATA = "../../data/toxigen/"
annotated_train, annotated_test = utils.prepare_datasets()
model = BertForSequenceClassification.from_pretrained(config.MODEL_NAME)
tokenizer = BertTokenizer.from_pretrained(config.TOKENIZER_NAME)

train_dataloader = utils.get_dataloader(annotated_train, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)
val_dataloader = utils.get_dataloader(annotated_test, tokenizer, config.MAX_LENGTH, config.BATCH_SIZE)
first_module_baseline = utils.FirstModuleBaseline(annotated_train["text"].tolist(), annotated_test["text"].tolist(), model, tokenizer)

In [3]:
scores = first_module_baseline.get_Bm25_scores()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anast\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anast\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\anast\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anast\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
scores.shape

torch.Size([940, 8960])

In [9]:
scores = first_module_baseline.get_FAISS_scores()
scores

KeyboardInterrupt: 

In [5]:
group_indices_train = annotated_train['target_group'].astype('category').cat.codes.tolist()
group_indices_val = annotated_test['target_group'].astype('category').cat.codes.tolist()

In [6]:
d3m_bm25 = utils.D3M(
        model=model,
        checkpoints=[],
        train_dataloader=train_dataloader,
        val_dataloader = val_dataloader,
        group_indices_train=group_indices_train,
        group_indices_val=group_indices_val,
        scores=scores,
        train_set_size=None,
        val_set_size=None,
        device="cpu")

In [7]:
debiased_train_indices = d3m_bm25.debias(num_to_discard=50)